In [10]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
import numpy as np
import open3d as o3d

from pathlib import Path
from skimage import io

from src.detection.points.orb import ORB
from src.detection.points.sift import SIFT
from src.matching.points.orb_matcher import ORBMatcher
from src.matching.points.sift_matcher import SIFTMatcher
from src.relative_pose_estimation.rgbd_point_pose_estimator import (
    RGBDPointPoseEstimator,
)
from src.geometry.transform import make_homogeneous_matrix
from src.keyframe_selector import EveryNthKeyframeSelector
from src.sensor.depth import DepthImage
from src.sensor.rgb import RGBImage
from src.sensor.rgbd import RGBDImage
from src.slam.frontend import PrimeSLAMFrontend

In [12]:
def get_point_cloud(points_3d):
    point_cloud = o3d.geometry.PointCloud()
    points = []

    for point in points_3d:
        points.append(point)

    point_cloud.points = o3d.utility.Vector3dVector(points)

    return point_cloud

In [13]:
images_path = Path("./rgb")
depth_path = Path("./depth")
intrinsics_path = Path("./intrinsics.txt")

images_paths = sorted(images_path.iterdir())
depth_paths = sorted(depth_path.iterdir())
intrinsics = make_homogeneous_matrix(np.genfromtxt(intrinsics_path))
depth_scaler = 5000
images_number = len(depth_paths)

In [14]:
features = "ORB"

if features == "ORB":
    extractor = ORB(nfeatures=1000)
    matcher = ORBMatcher()
else:
    extractor = SIFT()
    matcher = SIFTMatcher()

In [15]:
images = [io.imread(images_paths[i]) for i in range(images_number)]
depths = [io.imread(depth_paths[i]) for i in range(images_number)]

frames = [
    RGBDImage(RGBImage(img), DepthImage(depth, intrinsics, depth_scaler))
    for img, depth in zip(images, depths)
]
relative_pose_estimator = RGBDPointPoseEstimator(
    intrinsics, reprojection_threshold=30
)
keyframe_selector = EveryNthKeyframeSelector(n=1)

slam = PrimeSLAMFrontend(
    extractor,
    matcher,
    relative_pose_estimator,
    keyframe_selector,
    init_pose=np.eye(4),
)

for frame in frames:
    slam.process_frame(frame)

In [8]:
keyframes = slam.keyframes
abs_poses = [kf.world_to_camera_transform for kf in keyframes]
depths = [kf.sensor_measurement.depth for kf in keyframes]
features_batch = [kf.features for kf in keyframes]
keypoints_batch = [
    np.array([np.array([feature.x, feature.y]) for feature in features])
    for features in features_batch
]

keypoints_3D_batch = [
    depth.back_project_points(keypoints)
    for keypoints, depth in zip(keypoints_batch, depths)
]
clouds = [
    get_point_cloud(keypoints_3D).transform(np.linalg.inv(pose))
    for keypoints_3D, pose in zip(keypoints_3D_batch, abs_poses)
]
o3d.visualization.draw_geometries(clouds)